In [0]:
!apt-get update -qq 2>&1 > /dev/null
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse 

In [0]:
 from google.colab import drive
 drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import time
import urllib
from time import sleep
import pandas as pd 
import gc
import socket
from PIL import Image
import math
import os

curIdData = countImg = countAbort = 0

#get selected classes
dfClass = pd.read_csv("/content/drive/My Drive/Projects/BKSeeing/Tool/class-des-bkseeing-demo.csv")
numClass = len(dfClass)
print('Classes imported!')

dfBox = pd.read_csv(
    "/content/drive/My Drive/Projects/BKSeeing/Tool/train-annotations-bbox.csv",
    nrows=3000000)
numBox = len(dfBox)

dfBox = dfBox[['ImageID', 'LabelName']]
# print(dfBox)
print('Boxes imported!')
print(numClass, 'Classes')
print(numBox, 'Boxes')

#get dummy selected classes
dfCount = pd.read_csv("/content/drive/My Drive/Projects/BKSeeing/Tool/class-count-bkseeing-demo.csv")
dfTmp = pd.read_csv("/content/drive/My Drive/Projects/BKSeeing/Tool/class-count-bkseeing-demo.csv")

dfSort = pd.DataFrame()

curID = 0

folderNum = 1

#get the priority
print('Getting Priority!')
curPos = 0
cntTmp = 0
BoxName = dfBox['ImageID'][0]
for i in range(0, numBox-1):
  if i % 100000 == 0:
    print(i/100000, '%')   
  label = dfBox['LabelName'][i]
  for ii in range (0, numClass):
    if dfClass['LabelName'][ii] == label:
      cntTmp += 1
      break
  if BoxName != dfBox['ImageID'][i + 1]:
    dfSort.loc[curPos, 'ImageID'] = BoxName
    dfSort.loc[curPos, 'Sum'] = cntTmp
    cntTmp = 0
    curPos += 1
    BoxName = dfBox['ImageID'][i+1]
dfSort = dfSort.sort_values(by=['Sum'], ascending = False).reset_index(drop=True)
dfSort = dfSort.reset_index(drop=True)
numSort = len(dfSort)
print(dfSort)
dfSort.to_csv("/content/drive/My Drive/Projects/BKSeeing/Tool/Priority10mil.csv")
print('Images sorted!')


Classes imported!
Boxes imported!
8 Classes
3000000 Boxes
Getting Priority!
0.0 %
1.0 %
2.0 %
3.0 %
4.0 %
5.0 %
6.0 %
7.0 %
8.0 %
9.0 %
10.0 %
11.0 %
12.0 %
13.0 %
14.0 %
15.0 %
16.0 %
17.0 %
18.0 %
19.0 %
20.0 %
21.0 %
22.0 %
23.0 %
24.0 %
25.0 %
26.0 %
27.0 %
28.0 %
29.0 %
                 ImageID    Sum
0       05b0aefc92880a6c  175.0
1       16fabb214f6cdbb1  174.0
2       2f3ddc1f3383bd8e  169.0
3       11732b89775a33d9  153.0
4       0164fb7642c9b6fb  151.0
5       05924e7d3d1dd6c7  132.0
6       1d7382c365bdc58b  126.0
7       1cfea0fe8b47f138  122.0
8       01b881b13a5ef6c6  121.0
9       29fe3777920bec5f  114.0
10      12ed8de0f39a75e9  112.0
11      26937d30a600e286  109.0
12      2f103869d385a7b2  107.0
13      10a2d81e793551ff  103.0
14      021d884016cabbb1  101.0
15      0ea8ff7218cad0f4   97.0
16      0001a17a69ddeead   95.0
17      2a6fbfefa8868f90   95.0
18      13ac6637f48ffa12   95.0
19      05712a62d7569d71   94.0
20      2213240a23b434a6   91.0
21      107ab5fb3496

In [0]:
#@title Get images { form-width: "20%" }
import time
import urllib
from time import sleep
import pandas as pd 
import requests
import gc
import socket
from threading import Thread
from PIL import Image
import math
from bs4 import BeautifulSoup
import os

# socket.setdefaulttimeout(60)
global curIdData
global countImg 
global countAbort

curIdData = countImg = countAbort = 0

def printProgressBar(iteration,
                     total,
                     prefix='',
                     suffix='',
                     decimals=1,
                     length=100,
                     fill='█'):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(
        100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end='\r')
    # Print New Line on Complete
    if iteration == total:
        print()

class TooSlowException(Exception):
    pass

class CompleteDownload(Exception):
    pass

class AbortImage(Exception):
    pass

def getRow(width, height, i):
  output = ""
  j = i
  while (dfBox['ImageID'][i] == dfBox['ImageID'][j]) and (j > 0):
    ObjectName = 'Others'
    for ii in range(0, numClass):
        if dfBox['LabelName'][j] == dfClass['LabelName'][ii]:
          ObjectName = dfClass['Name'][ii]
          break
    if ObjectName == 'Woman':
      ObjectName = 'Man'
    if ObjectName == 'Desk':
      ObjectName = 'Table'
    XMin = max(0, math.floor(dfBox['XMin'][j] * width))
    XMax = min(width, math.ceil(dfBox['XMax'][j] * width))
    YMin = max(0, math.floor(dfBox['YMin'][j] * height))
    YMax = min(height, math.ceil(dfBox['YMax'][j] * height))
    IsTruncated = dfBox['IsTruncated'][j]
    output = output + (
    """<object>
        <name>%s</name>
        <pose>%s</pose>
        <truncated>%s</truncated>
        <difficult>%d</difficult>
        <bndbox>
            <xmin>%d</xmin>
            <ymin>%d</ymin>
            <xmax>%d</xmax>
            <ymax>%d</ymax>
        </bndbox>
    </object>
    """ % (
      ObjectName, 'Unspecified', IsTruncated, 0, XMin, YMin, XMax, YMax))
    # print(output)
    j -= 1
  return output+'\n'

def writeFile(imageName, imageFolder, imageInfo, imageInfoFolder, i):
  imagePath = imageFolder+imageName
  imageInfoPath = imageInfoFolder+imageInfo
  img = Image.open(imagePath)
  width, height = img.size
  # print(width, height)
  fi = open(imageInfoPath, 'w')
  fi.write(
  """<annotation>
    <folder>%s</folder>
    <filename>%s</filename>
    <path>%s</path>
    <source>
        <database>Unknown</database>
    </source>
    <size>
      <width>%d</width>
      <height>%d</height>
      <depth>3</depth>
    </size>
    <segmented>0</segmented>
    %s</annotation>"""
  %('data', imageName, imagePath, width, height, getRow(width, height, i)))
  fi.close()

def convertBToMb(bytes):
    """converts Bytes to Megabytes"""
    bytes = float(bytes)
    megabytes = bytes / 1048576
    return megabytes

def dlProgress(count, blockSize, totalSize):
    
    alreadyLoaded = count * blockSize
    timePassed = time.time() - startTime
    transferRate = convertBToMb(
        alreadyLoaded) / timePassed  # mbytes per second
    # transferRate *= 60 # mbytes per minute

    percent = int(alreadyLoaded * 100 / totalSize)
    # print('R:', transferRate)
    # print('T:', timePassed)
    # print('P:', percent)
    if percent >= 100:
        raise CompleteDownload


def DownloadFile(imageID, imageURL, i):
  global countImg 
  global countAbort  
  folderNum = (int)((countImg + 2000) / 2000)
  if(countImg%2000 == 0):
    print(folderNum)
  imageFolder = '/content/drive/My Drive/Projects/BKSeeing/data-ver-2.0/'
  imageInfoFolder = '/content/drive/My Drive/Projects/BKSeeing/data-ver-2.0/'
  imageFolder = "%sjpg%d/"%(imageFolder, folderNum)
  imageInfoFolder = "%sxml%d/"%(imageInfoFolder, folderNum)
  imageName = imageID+'.jpg'
  imageInfo = imageID+'.xml'
  global startTime
  startTime = time.time()
  r = requests.get(imageURL)
  f = open(imageFolder+imageName, 'wb')
  for chunk in r.iter_content(chunk_size=512 * 1024):
    if (chunk):
      f.write(chunk)
  f.close()
  if os.path.exists(imageFolder+imageName) and os.path.getsize(imageFolder+imageName) < 5000:
    os.remove(imageFolder+imageName)
    countAbort += 1
    return 0
  countImg += 1
  print(countImg, 'complete')
  writeFile(imageName, imageFolder, imageInfo, imageInfoFolder, i)
  return 1
  

dfImg = pd.read_csv(
    "/content/drive/My Drive/Projects/BKSeeing/Tool/train-images-boxable-with-rotation.csv")
dfImg.fillna(-99999, inplace=True)
dfImg = dfImg.sort_values(by=['ImageID']).reset_index(drop=True)
dfImg = dfImg.reset_index(drop=True)
numImg = len(dfImg)
print('Image URL imported!')

#get selected classes
dfClass = pd.read_csv("/content/drive/My Drive/Projects/BKSeeing/Tool/class-des-bkseeing-demo.csv")
numClass = len(dfClass)
print('Classes imported!')

dfBox = pd.read_csv(
    "/content/drive/My Drive/Projects/BKSeeing/Tool/train-annotations-bbox.csv",
    nrows=10000000)
numBox = len(dfBox)
print('Boxes imported!')
print(numImg, numClass, numBox)

#get dummy selected classes
dfCount = pd.read_csv("/content/drive/My Drive/Projects/BKSeeing/Tool/class-count-bkseeing-demo.csv")
dfTmp = pd.read_csv("/content/drive/My Drive/Projects/BKSeeing/Tool/class-count-bkseeing-demo.csv")

dfID = pd.read_csv("/content/drive/My Drive/Projects/BKSeeing/Tool/ID-origin.csv")

dfSort =  pd.read_csv("/content/drive/My Drive/Projects/BKSeeing/Tool/Priority10mil.csv")

curID = 0

printProgressBar(
    0, numBox, prefix='Progress:', suffix='Complete', decimals=3, length=50)

folderNum = 1

#get the priority
# print('Getting Priority!')
# curPos = 0
# cntTmp = 0
# for i in range(0, numBox-1):
#   if i % 1000 == 0:
#     printProgressBar(
#       i, numBox, prefix='Progress:', suffix='Complete', decimals=3, length=50)
#   for ii in range (0, numClass):
#     if dfClass['LabelName'][ii] == dfBox['LabelName'][i]:
#       cntTmp += 1
#       break
#   if dfBox['ImageID'][i] != dfBox['ImageID'][i + 1]:
#     dfSort.loc[curPos, 'ImageID'] = dfBox.loc[i, 'ImageID']
#     dfSort.loc[curPos, 'Sum'] = cntTmp
#     cntTmp = 0
#     curPos += 1
# dfSort = dfSort.sort_values(by=['Sum'], ascending = False).reset_index(drop=True)
# dfSort = dfSort.reset_index(drop=True)
# numSort = len(dfSort)
# print(dfSort)
# dfID.to_csv("/content/drive/My Drive/Projects/BKSeeing/Tool/Priority10mil.csv")
# print('Images sorted!')

print('Downloading Images!')
#get images
for i in range(0, numSort-1):
  if dfSort.loc[i, 'Sum'] == 0:
    break
  #get position of image in boxes file
  fileName = dfSort.loc[i, 'ImageID']
  imageID = fileName
  L = 0
  R = numBox + 1
  while R - L > 1:
    M = (R + L) // 2
    if dfBox['ImageID'][M] > fileName:
      R = M
    else:
      L = M
  for ii in range (0, numClass):
    dfTmp.loc[ii,'Quantity'] = 0
  #count number of each classes
  posImg = L
  j = L
  while j >= 0 and dfBox['ImageID'][j] == dfBox['ImageID'][posImg]:
    for ii in range (0, numClass):
      if dfClass['LabelName'][ii] == dfBox['LabelName'][j]:
        dfTmp.loc[ii, 'Quantity'] += 1
        break
    j -= 1
  #check number of boxes
  flag = 0
  for ii in range (0, numClass):
    if dfTmp.loc[ii, 'Quantity'] > 0:
      if dfCount.loc[ii, 'Quantity'] + dfTmp.loc[ii, 'Quantity'] < 4000:
        flag = 1
        break
  #image satisfied
  if flag == 1:
    #get image URL
    L = 0
    R = numImg + 1
    while R - L > 1:
      M = (R + L) // 2
      if dfImg['ImageID'][M] > dfBox['ImageID'][posImg]:
        R = M
      else:
        L = M
    imageURL = dfImg['Thumbnail300KURL'][L]
    if imageURL != -99999:
#       imageFolder = '/content/drive/My Drive/Projects/BKSeeing/data-ver-1.6/' 
#       if(len(os.listdir(imageFolder)) > 1900):
#         folderNum += 1
#         imageFolder = "%sjpg%d/"%(imageFolder, folderNum)
      my_thread = Thread(target = DownloadFile, args = (imageID, imageURL, posImg))
      my_thread.start()
      for ii in range (0, numClass):
        dfCount.loc[ii, 'Quantity'] += dfTmp.loc[ii, 'Quantity']
    
# for i in range(0, numBox - 1):
# #     print("i:%d"%(i))
#     for ii in range (0, numClass):
#       if dfClass['LabelName'][ii] == dfBox['LabelName'][i]:
#         dfTmp.loc[ii, 'Quantity'] += 1
#         break
#     if dfBox['ImageID'][i] != dfBox['ImageID'][i + 1]:
#       imageID = dfBox["ImageID"][i]

#       flag2 = 0
#       for ii in range (0, numClass):
#         if dfTmp.loc[ii, 'Quantity'] > 0:
#           if dfCount.loc[ii, 'Quantity'] + dfTmp.loc[ii, 'Quantity'] < 2000:
#             flag2 = 1
#             break
#       if flag2 == 1:
#         L = 0
#         R = numImg + 1
#         while R - L > 1:
#             M = (R + L) // 2
#             if dfImg['ImageID'][M] > dfBox['ImageID'][i]:
#                 R = M
#             else:
#                 L = M
# #             print(dfImg['ImageID'][L], dfBox['ImageID'][i])
#         imageURL = dfImg['Thumbnail300KURL'][L]
#         # print(imageURL)
#         if imageURL != -99999:  
# #               DownloadFile(imageID, imageURL, i)
#           # print(imageURL,'\n')
#           my_thread = Thread(target = DownloadFile, args = (imageID, imageURL, i, curID))
#           my_thread.start()
#           imageFolder = '/content/drive/My Drive/Projects/BKSeeing/data-ver-1.5/'
#           posID = max(1, math.ceil((curID+1)/2000))
#           imageFolder = "%sjpg%d/"%(imageFolder, posID)
#           if os.path.exists(imageFolder+imageID+'.jpg'):
#             for ii in range (0, numClass):
#               dfCount.loc[ii, 'Quantity'] += dfTmp.loc[ii, 'Quantity']
#             dfID.loc[curID, 'ImageName'] = imageID
#             curID += 1
#       for ii in range (0, numClass):
#           dfTmp.loc[ii,'Quantity'] = 0

#     if i % 100 == 0:
#         print("i:%d"%(i))
#         printProgressBar(
#             i,
#             numBox,
#             prefix='Progress:',
#             suffix='Complete',
#             decimals=3,
#             length=50)
#         gc.collect()
#     if i % 1000 == 0:
#         sleep(2)
# dfID.to_csv("/content/drive/My Drive/Projects/BKSeeing/Tool/ID.csv")
printProgressBar(
    numBox,
    numBox,
    prefix='Progress:',
    suffix='Complete',
    decimals=3,
    length=50)
print("Total:", countImg, "images")
print('Abort:', countAbort, 'images')
print(dfCount)


Image URL imported!
Classes imported!
Boxes imported!
1743042 8 10000000
1
1
1
1
1
1
1
1
1 complete
3 complete2
 complete
4 complete
5 complete
6 complete
7 complete
8 complete
9 complete
10 complete
11 complete
12 13 completecomplete

1514  completecomplete

16 complete
17 complete
18 complete
19 complete
20 complete
21 complete
22 complete
23 complete
25 24complete
 complete
26 complete
27 complete
28 complete
29 complete
30 complete
31 complete
32 complete
33 complete
343536  completecomplete

 complete
37 complete
38 complete
39 complete
40 complete
41 complete
42 complete
43 complete
44 complete
45 complete
46 complete
47 complete
48 complete
49 complete
50 complete
51 complete
52 complete
53 complete
54 complete
55 complete
56 complete
57 complete
58 complete
59 complete
60 complete
61 complete
62 complete
63 complete
64 complete
65 complete
66 complete67 complete68

 complete
69 complete
70 complete71
 complete
72 complete
73 complete
74 complete
75 complete
76 complete
77 compl

In [0]:
#@title Count number of boxes { form-width: "20%" }
import os
import pandas as pd

dfImg = pd.read_csv(
    "/content/drive/My Drive/Projects/BKSeeing/Tool/train-images-boxable-with-rotation.csv")
dfImg.fillna(-99999, inplace=True)
dfImg = dfImg.sort_values(by=['ImageID']).reset_index(drop=True)
dfImg = dfImg.reset_index(drop=True)
numImg = len(dfImg)
print('Image URL imported!')

dfClass = pd.read_csv("/content/drive/My Drive/Projects/BKSeeing/Tool/class-des-bkseeing-demo.csv")
numClass = len(dfClass)
print('Classes imported!')

dfBox = pd.read_csv(
    "/content/drive/My Drive/Projects/BKSeeing/Tool/train-annotations-bbox.csv",
    nrows=3000000)
print('Boxes imported!')
numBox = len(dfBox)

dfCount = pd.read_csv("/content/drive/My Drive/Projects/BKSeeing/Tool/class-count-bkseeing-demo.csv")
dfTmp = pd.read_csv("/content/drive/My Drive/Projects/BKSeeing/Tool/class-count-bkseeing-demo.csv")

dfID = pd.read_csv("/content/drive/My Drive/Projects/BKSeeing/Tool/ID.csv")

# curID = 0

# dir = '/content/drive/My Drive/Projects/BKSeeing/data-ver-1.2/'

# list = os.listdir(dir) 
# print('List complete!')
# print(len(list))

parDir = '/content/drive/My Drive/Projects/BKSeeing/data-ver-2.0'

c = 0
for i in range (1, 15):
  dir = "%s/jpg%d/"%(parDir, i)
  print(dir)
  list = os.listdir(dir)
  print("%d jpg files"%(len(list)))
  for fileFullName in list:
    if os.path.exists(dir+fileFullName) and fileFullName.endswith('.jpg') and os.path.getsize(dir+fileFullName) > 5000:
      fileName = os.path.splitext(fileFullName)[0]
      # print(fileName)
      L = 0
      R = numBox + 1
      while R - L > 1:
        M = (R + L) // 2
        if dfBox['ImageID'][M] > fileName:
          R = M
        else:
          L = M
      for i in range (0, numClass):
        dfTmp.loc[i,'Quantity'] = 0
      j = L
      while j >= 0 and dfBox['ImageID'][j] == dfBox['ImageID'][L]:
        for ii in range (0, numClass):
          if dfClass['LabelName'][ii] == dfBox['LabelName'][j]:
            dfTmp.loc[ii, 'Quantity'] += 1
            break
        j -= 1
      flag = 0
#       for i in range (0, numClass):
#         if dfCount.loc[i, 'Quantity'] + dfTmp.loc[i, 'Quantity'] > 2000:
#           flag = 1
#           break
      if flag == 0:
        c += 1
        for i in range (0, numClass):
          dfCount.loc[i, 'Quantity'] += dfTmp.loc[i, 'Quantity']
        # dfID.loc[curID, 'ImageName'] = fileName
#         curID += 1
#       curID += 1
#   if c == 10000:
#     break
# dfID.to_csv('/content/drive/My Drive/Projects/BKSeeing/Tool/ID.csv')
for i in range (1, 15):
  dir = "%s/xml%d/"%(parDir, i)
  print(dir)
  list = os.listdir(dir)
  print("%d xml files"%(len(list)))

print(dfCount)

Image URL imported!
Classes imported!
Boxes imported!
/content/drive/My Drive/Projects/BKSeeing/data-ver-2.0/jpg1/
2001 jpg files
/content/drive/My Drive/Projects/BKSeeing/data-ver-2.0/jpg2/
1999 jpg files
/content/drive/My Drive/Projects/BKSeeing/data-ver-2.0/jpg3/
2003 jpg files
/content/drive/My Drive/Projects/BKSeeing/data-ver-2.0/jpg4/
536 jpg files
/content/drive/My Drive/Projects/BKSeeing/data-ver-2.0/jpg5/
0 jpg files
/content/drive/My Drive/Projects/BKSeeing/data-ver-2.0/jpg6/
0 jpg files
/content/drive/My Drive/Projects/BKSeeing/data-ver-2.0/jpg7/
0 jpg files
/content/drive/My Drive/Projects/BKSeeing/data-ver-2.0/jpg8/
0 jpg files
/content/drive/My Drive/Projects/BKSeeing/data-ver-2.0/jpg9/
0 jpg files
/content/drive/My Drive/Projects/BKSeeing/data-ver-2.0/jpg10/
0 jpg files
/content/drive/My Drive/Projects/BKSeeing/data-ver-2.0/jpg11/
0 jpg files
/content/drive/My Drive/Projects/BKSeeing/data-ver-2.0/jpg12/
0 jpg files
/content/drive/My Drive/Projects/BKSeeing/data-ver-2.0/j

In [0]:
#@title Make folder to store images { run: "auto", form-width: "20%" }
import os

dirName = '/content/drive/My Drive/Projects/BKSeeing/data-ver-2.0'

if not os.path.exists(dirName):
  os.mkdir(dirName)
  print("Directory " , dirName ,  " Created ")

for i in range (21, 40):
  dir = "%s/jpg%d"%(dirName, i)
  if not os.path.exists(dir):
      os.mkdir(dir)
      print("Directory " , dir ,  " Created ")
  else:    
      print("Directory " , dir,  " already exists")
  dir = "%s/xml%d"%(dirName, i)
  if not os.path.exists(dir):
      os.mkdir(dir)
      print("Directory " , dir ,  " Created ")
  else:    
      print("Directory " , dir ,  " already exists")
